# Demo for the 2018 BHI & BSN Data Challenge

In [1]:
# Import libraries
from tableone import TableOne
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import psycopg2
import getpass
%matplotlib inline

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'public, mimiciii'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

Password:········


In [4]:
# Run query and assign the results to a Pandas DataFrame
# Requires the icustay_detail view from:
# https://github.com/MIT-LCP/mimic-code/tree/master/concepts/demographics
# And the OASIS score from:
# https://github.com/MIT-LCP/mimic-code/tree/master/concepts/severityscores
query = \
"""
WITH first_icu AS (
    SELECT i.subject_id, i.hadm_id, i.icustay_id, i.gender, i.admittime admittime_hospital, 
      i.dischtime dischtime_hospital, i.los_hospital, i.age, i.admission_type, 
      i.hospital_expire_flag, i.intime intime_icu, i.outtime outtime_icu, i.los_icu, 
      s.first_careunit
    FROM icustay_detail i
    LEFT JOIN icustays s
    ON i.icustay_id = s.icustay_id
    WHERE i.hospstay_seq = 1
      AND i.icustay_seq = 1
      AND i.age >= 16
)
SELECT f.*, o.icustay_expire_flag, o.oasis, o.oasis_prob
FROM first_icu f
LEFT JOIN oasis o
ON f.icustay_id = o.icustay_id;
"""

data = pd.read_sql_query(query,con)

# Display the first few rows of the data

In [5]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'first_careunit', 'icustay_expire_flag', 'oasis', 'oasis_prob'],
      dtype='object')

In [ ]:
data.head()

# Add day of week to DataFrame

In [ ]:
data['admitday_hospital'] = data.admittime_hospital.dt.weekday_name
data['dischday_hospital'] = data.dischtime_hospital.dt.weekday_name
data['inday_icu'] = data.intime_icu.dt.weekday_name
data['outday_icu'] = data.outtime_icu.dt.weekday_name
data.head()

# Table 1 summary statistics

In [8]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'first_careunit', 'icustay_expire_flag', 'oasis', 'oasis_prob',
       'admitday_hospital', 'dischday_hospital', 'inday_icu', 'outday_icu'],
      dtype='object')

In [11]:
columns = ['gender', 'los_hospital', 'age', 'admission_type', 'hospital_expire_flag', 
           'los_icu','icustay_expire_flag', 'oasis', 'oasis_prob', 'first_careunit']

groupby = 'inday_icu'

pval = False

categorical = ['gender','admission_type','hospital_expire_flag','icustay_expire_flag',
               'first_careunit']

t = TableOne(data, columns=columns, categorical=categorical, groupby=groupby, pval=pval)
t.tableone

Grouped by inday_icu                 \
                                             Friday         Monday   
variable             level                                           
n                                              6263           6097   
admission_type       ELECTIVE          1016 (16.22)   1265 (20.75)   
                     EMERGENCY         5118 (81.72)   4687 (76.87)   
                     URGENT              129 (2.06)     145 (2.38)   
age                                   74.56 (53.22)  73.13 (51.37)   
first_careunit       CCU                838 (13.38)    918 (15.06)   
                     CSRU              1416 (22.61)   1632 (26.77)   
                     MICU              2139 (34.15)   1940 (31.82)   
                     SICU              1044 (16.67)    865 (14.19)   
                     TSICU              826 (13.19)    742 (12.17)   
gender               F                  2662 (42.5)   2559 (41.97)   
                     M                  3601 (57.5)   3538 (58.03)   
hospital_expire_flag 0                 5576 (89.03)   5468 (89.68)   
                     1                  687 (10.97)    629 (10.32)   
icustay_expire_flag  0                  5768 (92.1)   5650 (92.67)   
                     1                    495 (7.9)     447 (7.33)   
los_hospital                          10.23 (11.61)    9.71 (9.92)   
los_icu                                 4.15 (6.19)    3.83 (5.52)   
oasis                                  31.17 (9.02)   31.21 (8.77)   
oasis_prob                              0.14 (0.14)    0.14 (0.14)   

                                                                             \
                                     Saturday         Sunday       Thursday   
variable             level                                                    
n                                        4235           3960           5876   
admission_type       ELECTIVE      162 (3.83)     101 (2.55)     999 (17.0)   
                     EMERGENCY   3852 (90.96)   3681 (92.95)   4746 (80.77)   
                     URGENT        221 (5.22)     178 (4.49)     131 (2.23)   
age                             73.92 (58.58)  75.26 (60.66)  75.51 (55.46)   
first_careunit       CCU          695 (16.41)    621 (15.68)    850 (14.47)   
                     CSRU           237 (5.6)      194 (4.9)   1282 (21.82)   
                     MICU        1765 (41.68)   1706 (43.08)   2020 (34.38)   
                     SICU         743 (17.54)    743 (18.76)    996 (16.95)   
                     TSICU        795 (18.77)    696 (17.58)    728 (12.39)   
gender               F           1857 (43.85)   1736 (43.84)    2603 (44.3)   
                     M           2378 (56.15)   2224 (56.16)    3273 (55.7)   
hospital_expire_flag 0           3658 (86.38)   3388 (85.56)   5202 (88.53)   
                     1            577 (13.62)    572 (14.44)    674 (11.47)   
icustay_expire_flag  0           3811 (89.99)    3548 (89.6)   5399 (91.88)   
                     1            424 (10.01)     412 (10.4)     477 (8.12)   
los_hospital                    10.00 (10.84)   9.82 (10.70)   9.88 (10.90)   
los_icu                           4.42 (6.58)    4.44 (6.29)    3.96 (6.12)   
oasis                            31.49 (9.27)   32.07 (9.02)   31.10 (8.82)   
oasis_prob                        0.15 (0.15)    0.16 (0.15)    0.14 (0.14)   

                                                                     
                                      Tuesday      Wednesday isnull  
variable             level                                           
n                                        6141           5985         
admission_type       ELECTIVE    1292 (21.04)   1243 (20.77)      0  
                     EMERGENCY    4704 (76.6)   4600 (76.86)         
                     URGENT        145 (2.36)     142 (2.37)         
age                             75.48 (55.22)  74.16 (53.88)      0  
first_careunit       CCU          919 (14.

In [10]:
data.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime_hospital',
       'dischtime_hospital', 'los_hospital', 'age', 'admission_type',
       'hospital_expire_flag', 'intime_icu', 'outtime_icu', 'los_icu',
       'first_careunit', 'icustay_expire_flag', 'oasis', 'oasis_prob',
       'admitday_hospital', 'dischday_hospital', 'inday_icu', 'outday_icu'],
      dtype='object', name='variable')